In [50]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Dataset MNIST

El dataset **MNIST** consiste en imágenes de números manuscritos entre 0 y 9. Las imágenes tienen dimensiones de 28x28 pixeles y cada pixel está representado por un valor de intensidad en escala de grises. El conjunto de entrenamiento consiste en 60000 dígitos y el conjunto de prueba de 10000.

### Implemente una CNN para obtener un desempeño de al menos 99% en el conjunto de prueba

In [60]:
#Importamos las librerias y la base de datos MNIST
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

In [61]:
# Cargar el dataset MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [62]:
# Preprocesamos los datos
# El reshape se realiza para agregar una dimension de color al dataset, en este caso es 1 porque es escala de grises
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
# Ahora se normalizan los valores de los pixeles de 0 a 1, se divide por 255 que es el valor maximo de un pixel
train_images, test_images = train_images / 255.0, test_images / 255.0

In [63]:
# Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)
datagen.fit(train_images)

In [64]:
# Definimos la red neuronal convolucional
# Sequential permite crear modelos capa por capa de forma secuencial
model = models.Sequential([
    # Capa convolucional, 32 filtros de 3x3, funcion de activacion relu, input_shape es el tamaño de la imagen
    # Kerner_regularizer se utiliza para evitar el overfitting, penaliza los pesos grandes
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # Capa de pooling, se utiliza para reducir las dimensiones espaciales de salida, por ende el número de parámetros de la red
    layers.MaxPooling2D((2, 2)),
    # Capa convolucional, 16 filtros de 3x3, funcion de activacion relu
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # La repetición de los parámetros de arriba se debe a que se busca aumentar la profundidad de la red
    # Capa flatten, se utiliza para aplanar la entrada, se utiliza para pasar de una capa convolucional a una densa
    layers.Flatten(),
    layers.Dropout(0.3),
    # Capa densa, 32 neuronas, funcion de activacion relu
    layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # Capa densa, 10 neuronas, funcion de activacion softmax, esta última se utiliza para problemas de clasificación multiclase
    layers.Dense(10, activation='softmax')
])

In [65]:
# Early stopping, callback de la función fit, se utiliza para detener el entrenamiento cuando se alcanza un valor de perdida
# no tolerable, en este caso se utiliza para evitar el overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5)

In [66]:
# Compilamos el modelo
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
# sparse_categorical_crossentropy se utiliza para problemas de clasificación multiclase
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [67]:
# Entrenamos el modelo
model.fit(datagen.flow(train_images, train_labels, batch_size=32), epochs=30, callbacks=[early_stopping])

Epoch 1/30
1875/1875 [==============================] - 13s 7ms/step - loss: 2.0536 - accuracy: 0.6066
Epoch 2/30
1875/1875 [==============================] - 12s 6ms/step - loss: 1.0962 - accuracy: 0.8547
Epoch 3/30
1875/1875 [==============================] - 12s 6ms/step - loss: 0.8551 - accuracy: 0.8944
Epoch 4/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.7294 - accuracy: 0.9087
Epoch 5/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.6528 - accuracy: 0.9180
Epoch 6/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.6025 - accuracy: 0.9235
Epoch 7/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.5670 - accuracy: 0.9263
Epoch 8/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.5463 - accuracy: 0.9285
Epoch 9/30
1875/1875 [==============================] - 12s 7ms/step - loss: 0.5239 - accuracy: 0.9318
Epoch 10/30
1875/1875 [==============================] - 12s 7ms/step - l

In [68]:
# Evaluamos el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy en el conjunto de prueba: {test_acc*100:.2f}%')

313/313 [==============================] - 1s 2ms/step - loss: 0.2973 - accuracy: 0.9779
Accuracy en el conjunto de prueba: 97.79%
